In [3]:
!pip install -U xgboost polars optuna catboost lightgbm


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import polars as pl
from sklearn.model_selection import GroupShuffleSplit
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

# Constants
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Loading raw data...")
# Load data using polars for efficiency
train = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet').drop('__index_level_0__')
test = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet').drop('__index_level_0__').with_columns(pl.lit(0, dtype=pl.Int64).alias("selected"))

data_raw = pl.concat((train, test))

print(f"Train shape: {train.shape}, Test shape: {test.shape}")
print(f"Unique ranker_ids in train: {train['ranker_id'].n_unique():,}")
print(f"Selected rate: {train['selected'].mean():.3f}")

def dur_to_min(col):
    """More efficient duration to minutes converter"""
    # Extract days and time parts in one pass
    days = col.str.extract(r"^(\d+)\.", 1).cast(pl.Int64).fill_null(0) * 1440
    time_str = pl.when(col.str.contains(r"^\d+\.")).then(col.str.replace(r"^\d+\.", "")).otherwise(col)
    hours = time_str.str.extract(r"^(\d+):", 1).cast(pl.Int64).fill_null(0) * 60
    minutes = time_str.str.extract(r":(\d+):", 1).cast(pl.Int64).fill_null(0)
    return (days + hours + minutes).fill_null(0)

print("Starting feature engineering...")
df = data_raw.clone()

# Process duration columns
dur_cols = ["legs0_duration", "legs1_duration"] + [f"legs{l}_segments{s}_duration" for l in (0, 1) for s in (0, 1)]
dur_exprs = [dur_to_min(pl.col(c)).alias(c) for c in dur_cols if c in df.columns]

# Apply duration transformations first
if dur_exprs:
    df = df.with_columns(dur_exprs)

# Precompute marketing carrier columns check
mc_cols = [f'legs{l}_segments{s}_marketingCarrier_code' for l in (0, 1) for s in range(4)]
mc_exists = [col for col in mc_cols if col in df.columns]

# Combine all initial transformations
df = df.with_columns([
    # Price features
    (pl.col("totalPrice") / (pl.col("taxes") + 1)).alias("price_per_tax"),
    (pl.col("taxes") / (pl.col("totalPrice") + 1)).alias("tax_rate"),
    pl.col("totalPrice").log1p().alias("log_price"),
    
    # Duration features
    (pl.col("legs0_duration").fill_null(0) + pl.col("legs1_duration").fill_null(0)).alias("total_duration"),
    pl.when(pl.col("legs1_duration").fill_null(0) > 0)
        .then(pl.col("legs0_duration") / (pl.col("legs1_duration") + 1))
        .otherwise(1.0).alias("duration_ratio"),
    
    # Trip type
    (pl.col("legs1_duration").is_null() | 
     (pl.col("legs1_duration") == 0) | 
     pl.col("legs1_segments0_departureFrom_airport_iata").is_null()).cast(pl.Int32).alias("is_one_way"),
    
    # Total segments count
    (pl.sum_horizontal(pl.col(col).is_not_null().cast(pl.UInt8) for col in mc_exists) 
     if mc_exists else pl.lit(0)).alias("l0_seg"),
    
    # FF features
    (pl.col("frequentFlyer").fill_null("").str.count_matches("/") + 
     (pl.col("frequentFlyer").fill_null("") != "").cast(pl.Int32)).alias("n_ff_programs"),
    
    # Binary features
    pl.col("corporateTariffCode").is_not_null().cast(pl.Int32).alias("has_corporate_tariff"),
    (pl.col("pricingInfo_isAccessTP") == 1).cast(pl.Int32).alias("has_access_tp"),
    
    # Cancellation/Exchange rules
    (
        (pl.col("miniRules0_monetaryAmount") == 0)
        & (pl.col("miniRules0_statusInfos") == 1)
    ).cast(pl.Int8).alias("free_cancel"),
    (
        (pl.col("miniRules1_monetaryAmount") == 0)
        & (pl.col("miniRules1_statusInfos") == 1)
    ).cast(pl.Int8).alias("free_exchange"),

    # Routes & carriers
    pl.col("searchRoute").is_in(["MOWLED/LEDMOW", "LEDMOW/MOWLED", "MOWLED", "LEDMOW"])
        .cast(pl.Int32).alias("is_popular_route"),
    
    # Cabin
    pl.mean_horizontal(["legs0_segments0_cabinClass", "legs1_segments0_cabinClass"]).alias("avg_cabin_class"),
    (pl.col("legs0_segments0_cabinClass").fill_null(0) - 
     pl.col("legs1_segments0_cabinClass").fill_null(0)).alias("cabin_class_diff"),
])

# Segment counts - more efficient
seg_exprs = []
for leg in (0, 1):
    seg_cols = [f"legs{leg}_segments{s}_duration" for s in range(4) if f"legs{leg}_segments{s}_duration" in df.columns]
    if seg_cols:
        seg_exprs.append(
            pl.sum_horizontal(pl.col(c).is_not_null() for c in seg_cols)
                .cast(pl.Int32).alias(f"n_segments_leg{leg}")
        )
    else:
        seg_exprs.append(pl.lit(0).cast(pl.Int32).alias(f"n_segments_leg{leg}"))

# Add segment-based features
df = df.with_columns(seg_exprs)

# Then use them for derived features
df = df.with_columns([
    (pl.col("n_segments_leg0") + pl.col("n_segments_leg1")).alias("total_segments"),
    (pl.col("n_segments_leg0") == 1).cast(pl.Int32).alias("is_direct_leg0"),
    pl.when(pl.col("is_one_way") == 1).then(0)
        .otherwise((pl.col("n_segments_leg1") == 1).cast(pl.Int32)).alias("is_direct_leg1"),
])

# More derived features
df = df.with_columns([
    (pl.col("is_direct_leg0") & pl.col("is_direct_leg1")).cast(pl.Int32).alias("both_direct"),
    ((pl.col("isVip") == 1) | (pl.col("n_ff_programs") > 0)).cast(pl.Int32).alias("is_vip_freq"),
    pl.col("Id").count().over("ranker_id").alias("group_size"),
])

# Add major carrier flag if column exists
if "legs0_segments0_marketingCarrier_code" in df.columns:
    df = df.with_columns(
        pl.col("legs0_segments0_marketingCarrier_code").is_in(["SU", "S7"])
            .cast(pl.Int32).alias("is_major_carrier")
    )
else:
    df = df.with_columns(pl.lit(0).alias("is_major_carrier"))

df = df.with_columns(pl.col("group_size").log1p().alias("group_size_log"))

# Time features - batch process
time_exprs = []
for col in ("legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"):
    if col in df.columns:
        dt = pl.col(col).str.to_datetime(strict=False)
        h = dt.dt.hour().fill_null(12)
        time_exprs.extend([
            h.alias(f"{col}_hour"),
            dt.dt.weekday().fill_null(0).alias(f"{col}_weekday"),
            (((h >= 6) & (h <= 9)) | ((h >= 17) & (h <= 20))).cast(pl.Int32).alias(f"{col}_business_time")
        ])
if time_exprs:
    df = df.with_columns(time_exprs)

# Price and duration basic ranks
rank_exprs = []
for col, alias in [("totalPrice", "price"), ("total_duration", "duration")]:
    rank_exprs.append(pl.col(col).rank().over("ranker_id").alias(f"{alias}_rank"))

# Price-specific features
price_exprs = [
    (pl.col("totalPrice").rank("average").over("ranker_id") / 
     pl.col("totalPrice").count().over("ranker_id")).alias("price_pct_rank"),
    (pl.col("totalPrice") == pl.col("totalPrice").min().over("ranker_id")).cast(pl.Int32).alias("is_cheapest"),
    ((pl.col("totalPrice") - pl.col("totalPrice").median().over("ranker_id")) / 
     (pl.col("totalPrice").std().over("ranker_id") + 1)).alias("price_from_median"),
    (pl.col("l0_seg") == pl.col("l0_seg").min().over("ranker_id")).cast(pl.Int32).alias("is_min_segments"),
]

# Apply initial ranks
df = df.with_columns(rank_exprs + price_exprs)

# Cheapest direct - more efficient
direct_cheapest = (
    df.filter(pl.col("is_direct_leg0") == 1)
    .group_by("ranker_id")
    .agg(pl.col("totalPrice").min().alias("min_direct"))
)

df = df.join(direct_cheapest, on="ranker_id", how="left").with_columns(
    ((pl.col("is_direct_leg0") == 1) & 
     (pl.col("totalPrice") == pl.col("min_direct"))).cast(pl.Int32).fill_null(0).alias("is_direct_cheapest")
).drop("min_direct")

# Popularity features - efficient join
df = (
    df.join(
        train.group_by('legs0_segments0_marketingCarrier_code').agg(pl.mean('selected').alias('carrier0_pop')),
        on='legs0_segments0_marketingCarrier_code', 
        how='left'
    )
    .join(
        train.group_by('legs1_segments0_marketingCarrier_code').agg(pl.mean('selected').alias('carrier1_pop')),
        on='legs1_segments0_marketingCarrier_code', 
        how='left'
    )
    .with_columns([
        pl.col('carrier0_pop').fill_null(0.0),
        pl.col('carrier1_pop').fill_null(0.0),
    ])
)

# Step 1: Add independent features
df = df.with_columns([
    # Carrier popularity
    (pl.col('carrier0_pop') * pl.col('carrier1_pop')).alias('carrier_pop_product'),
])

# Business traveler and meeting-friendly features
df = df.with_columns([
    (
        # Policy compliance (25% weight)
        (pl.col("pricingInfo_isAccessTP") * 0.25) +
        # Direct flights (25% weight)
        (pl.col("is_direct_leg0") * 0.25) +
        # Business-hour departures/arrivals (25% weight)
        ((pl.col("legs0_departureAt_business_time") + pl.col("legs1_departureAt_business_time")) * 0.125) +
        # VIP preference for business class (25% weight)
        ((pl.col("isVip") == 1) & (pl.col("avg_cabin_class") >= 1.5)).cast(pl.Int8) * 0.25
    ).alias("business_traveler_perfect_match"),

    # Timezone diff only
    (pl.col("legs0_arrivalAt_hour") - pl.col("legs0_departureAt_hour") -
     (pl.col("legs0_duration") / 60)).alias("timezone_diff_leg0"),
])

df = df.with_columns([
    (
        (pl.col("is_one_way") == 0) &  # Round-trip
        (pl.col("legs0_arrivalAt_hour") >= 8) &  # Arrive by morning
        (pl.col("legs1_departureAt_hour") <= 18) &  # Return by evening
        (pl.col("timezone_diff_leg0").abs() < 3)   # Minimal jetlag
    ).cast(pl.Int8).alias("meeting_friendly_itinerary")
])

# Additional temporal features
df = df.with_columns([
    # Days to departure
    (pl.col("legs0_departureAt").str.to_datetime(strict=False) - 
     pl.col("requestDate")).dt.total_days().fill_null(0).alias("days_to_departure"),
    
    pl.col("requestDate").dt.month().fill_null(6).alias("request_month"),
])

# Seasonal features
df = df.with_columns([
    ((pl.col("request_month") >= 6) & (pl.col("request_month") <= 8)).cast(pl.Int8).alias("is_summer_travel"),
    ((pl.col("request_month") >= 12) | (pl.col("request_month") <= 2)).cast(pl.Int8).alias("is_winter_travel"),
    ((pl.col("request_month").is_in([12, 1, 7, 8]))).cast(pl.Int8).alias("is_holiday_season"),
])

# Baggage features
df = df.with_columns([
    (pl.col("legs0_segments0_baggageAllowance_quantity").fill_null(0) + 
     pl.col("legs1_segments0_baggageAllowance_quantity").fill_null(0)).alias("baggage_total"),
    (pl.col("miniRules0_monetaryAmount").fill_null(0) + 
     pl.col("miniRules1_monetaryAmount").fill_null(0)).alias("total_fees"),
])

df = df.with_columns([
    (pl.col("baggage_total") > 0).cast(pl.Int32).alias("has_baggage"),
    (pl.col("total_fees") > 0).cast(pl.Int32).alias("has_fees"),
    (pl.col("total_fees") / (pl.col("totalPrice") + 1)).alias("fee_rate"),
])

print("Feature engineering completed. Filling missing values...")

# Fill missing values
data = df.with_columns(
    [pl.col(c).fill_null(0) for c in df.select(pl.selectors.numeric()).columns] +
    [pl.col(c).fill_null("missing") for c in df.select(pl.selectors.string()).columns]
)

# Define categorical features
cat_features = [
    'nationality', 'searchRoute', 'corporateTariffCode',
    'bySelf', 'sex', 'companyID',
    # Leg 0 segments 0-1
    'legs0_segments0_aircraft_code', 'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_marketingCarrier_code', 'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_flightNumber',
    'legs0_segments1_aircraft_code', 'legs0_segments1_arrivalTo_airport_city_iata',
    'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_departureFrom_airport_iata',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments1_operatingCarrier_code',
    'legs0_segments1_flightNumber',
    # Leg 1 segments 0-1
    'legs1_segments0_aircraft_code', 'legs1_segments0_arrivalTo_airport_city_iata',
    'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments0_departureFrom_airport_iata',
    'legs1_segments0_marketingCarrier_code', 'legs1_segments0_operatingCarrier_code',
    'legs1_segments0_flightNumber',
    'legs1_segments1_aircraft_code', 'legs1_segments1_arrivalTo_airport_city_iata',
    'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments1_departureFrom_airport_iata',
    'legs1_segments1_marketingCarrier_code', 'legs1_segments1_operatingCarrier_code',
    'legs1_segments1_flightNumber'
]

# Columns to exclude (uninformative or problematic)
exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate',
    'legs0_departureAt', 'legs0_arrivalAt', 'legs1_departureAt', 'legs1_arrivalAt',
    'miniRules0_percentage', 'miniRules1_percentage',  # >90% missing
    'frequentFlyer',  # Already processed
    # Exclude constant columns
    'pricingInfo_passengerCount', 'bySelf'
]

# Exclude high missing rate columns
for leg in [0, 1]:
    for seg in [0, 1]:
        if seg == 0:
            suffixes = ["seatsAvailable"]
        else:
            suffixes = [
                "cabinClass", "seatsAvailable", "baggageAllowance_quantity",
                "baggageAllowance_weightMeasurementType", "aircraft_code",
                "arrivalTo_airport_city_iata", "arrivalTo_airport_iata",
                "departureFrom_airport_iata", "flightNumber",
                "marketingCarrier_code", "operatingCarrier_code",
            ]
        for suffix in suffixes:
            exclude_cols.append(f"legs{leg}_segments{seg}_{suffix}")

# Exclude segment 2-3 columns (>98% missing)
for leg in [0, 1]:
    for seg in [2, 3]:
        for suffix in ['aircraft_code', 'arrivalTo_airport_city_iata', 'arrivalTo_airport_iata',
                      'baggageAllowance_quantity', 'baggageAllowance_weightMeasurementType',
                      'cabinClass', 'departureFrom_airport_iata', 'duration', 'flightNumber',
                      'marketingCarrier_code', 'operatingCarrier_code', 'seatsAvailable']:
            exclude_cols.append(f'legs{leg}_segments{seg}_{suffix}')

feature_cols = [col for col in data.columns if col not in exclude_cols]
cat_features_final = [col for col in cat_features if col in feature_cols]

print(f"Using {len(feature_cols)} features ({len(cat_features_final)} categorical)")

print("Splitting data...")
# Convert to pandas for splitting operations
data_pd = data.to_pandas()

# Split into train and test
n_train = train.shape[0]
train_data = data_pd.iloc[:n_train].copy()
test_data = data_pd.iloc[n_train:].copy()

# Group-wise train/val split ensuring 85% groups in train, 15% in val
unique_ranker_ids = train_data['ranker_id'].unique()
n_groups = len(unique_ranker_ids)
n_train_groups = int(n_groups * 0.85)

np.random.shuffle(unique_ranker_ids)
train_groups = unique_ranker_ids[:n_train_groups]
val_groups = unique_ranker_ids[n_train_groups:]

train_split = train_data[train_data['ranker_id'].isin(train_groups)].copy()
val_split = train_data[train_data['ranker_id'].isin(val_groups)].copy()

print(f"Train split: {len(train_split):,} rows ({len(train_groups):,} groups)")
print(f"Val split: {len(val_split):,} rows ({len(val_groups):,} groups)")
print(f"Test data: {len(test_data):,} rows")

# Save processed files
print("Saving processed files...")
train_data[feature_cols + ['selected', 'ranker_id']].to_parquet('train_processed.parquet', index=False)
test_data[feature_cols + ['ranker_id', 'Id']].to_parquet('test_processed.parquet', index=False)
train_split[feature_cols + ['selected', 'ranker_id']].to_parquet('train_split.parquet', index=False)
val_split[feature_cols + ['selected', 'ranker_id']].to_parquet('val_split.parquet', index=False)

# Save feature lists and metadata
with open('features.pkl', 'wb') as f:
    pickle.dump({
        'feature_cols': feature_cols,
        'cat_features_final': cat_features_final,
        'train_groups': train_groups,
        'val_groups': val_groups
    }, f)

print("✅ Data preprocessing completed successfully!")
print(f"Final dataset shape - Train: {train_data.shape}, Test: {test_data.shape}")
print(f"Features saved: {len(feature_cols)} total, {len(cat_features_final)} categorical")

gc.collect()

Loading raw data...
Train shape: (18145372, 126), Test shape: (6897776, 126)
Unique ranker_ids in train: 105,539
Selected rate: 0.006
Starting feature engineering...
Feature engineering completed. Filling missing values...
Using 100 features (19 categorical)
Splitting data...
Train split: 15,435,234 rows (89,708 groups)
Val split: 2,710,138 rows (15,831 groups)
Test data: 6,897,776 rows
Saving processed files...
✅ Data preprocessing completed successfully!
Final dataset shape - Train: (18145372, 186), Test: (6897776, 186)
Features saved: 100 total, 19 categorical


54